In [14]:
from google.colab import drive 
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [15]:
import os
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
import cv2
from google.colab.patches import cv2_imshow
import PIL.ImageOps  
import numpy as np

In [16]:
FJoin = os.path.join

In [17]:
f_data_final = ['a', 'à', 'á', 'â', 'ầ', 'ấ', 'ẫ', 'ẩ', 'ã', 'ă', 'ằ', 'ắ', 'ẵ', 'ẳ', 'ả', 'ạ', 'ậ', 'ặ', 'b', 'c', 'd', 'e', 'è', 'é', 'ê', 'ề', 'ế', 'ễ', 'ể', 'ẽ', 'ẻ', 'ẹ', 'ệ', 'g', 'h', 'i', 'ì', 'í', 'ĩ', 'ỉ', 'ị', 'k', 'l', 'm', 'n', 'o', 'ò', 'ó', 'ô', 'ồ', 'ố', 'ỗ', 'ổ', 'õ', 'ỏ', 'ơ', 'ờ', 'ớ', 'ỡ', 'ở', 'ợ', 'ọ', 'ộ', 'p', 'q', 'r', 's', 't', 'u', 'ù', 'ú', 'ũ', 'ủ', 'ư', 'ừ', 'ứ', 'ữ', 'ử', 'ự', 'ụ', 'v', 'x', 'y', 'ỳ', 'ý', 'ỹ', 'ỷ', 'ỵ', 'đ']
print(f_data_final)

['a', 'à', 'á', 'â', 'ầ', 'ấ', 'ẫ', 'ẩ', 'ã', 'ă', 'ằ', 'ắ', 'ẵ', 'ẳ', 'ả', 'ạ', 'ậ', 'ặ', 'b', 'c', 'd', 'e', 'è', 'é', 'ê', 'ề', 'ế', 'ễ', 'ể', 'ẽ', 'ẻ', 'ẹ', 'ệ', 'g', 'h', 'i', 'ì', 'í', 'ĩ', 'ỉ', 'ị', 'k', 'l', 'm', 'n', 'o', 'ò', 'ó', 'ô', 'ồ', 'ố', 'ỗ', 'ổ', 'õ', 'ỏ', 'ơ', 'ờ', 'ớ', 'ỡ', 'ở', 'ợ', 'ọ', 'ộ', 'p', 'q', 'r', 's', 't', 'u', 'ù', 'ú', 'ũ', 'ủ', 'ư', 'ừ', 'ứ', 'ữ', 'ử', 'ự', 'ụ', 'v', 'x', 'y', 'ỳ', 'ý', 'ỹ', 'ỷ', 'ỵ', 'đ']


In [18]:
data_train_drive_link = '/content/drive/MyDrive/Dataframe_training_final_CNN.csv'
label_train_drive_link = '/content/drive/MyDrive/Labelframe_training_final_CNN.csv'

data_test_drive_link = '/content/drive/MyDrive/Dataframe_testing_final_CNN.csv'
label_test_drive_link = '/content/drive/MyDrive/Labelframe_testing_final_CNN.csv'

In [19]:
def read_data(link_data, link_label):
    data = pd.read_csv(link_data, header=None)
    label = pd.read_csv(link_label, header=None)
    return np.array(data), np.array(label).ravel()

In [20]:
data_train, label_train = read_data(data_train_drive_link, label_train_drive_link)
data_test, label_test = read_data(data_test_drive_link, label_test_drive_link)

In [21]:
from sklearn.metrics import accuracy_score, classification_report, recall_score, f1_score,precision_score, confusion_matrix
from sklearn.preprocessing import MinMaxScaler
from sklearn import svm
from sklearn import metrics
import math
from sklearn.neural_network import MLPClassifier

In [22]:
def change_to_char(num):
  listchar = ['a', 'à', 'á', 'â', 'ầ', 'ấ', 'ẫ', 'ẩ', 'ã', 'ă', 'ằ', 'ắ', 'ẵ', 'ẳ', 'ả', 'ạ', 'ậ', 'ặ', 'b', 'c', 'd', 'e', 'è', 'é', 'ê', 'ề', 'ế', 'ễ', 'ể', 'ẽ', 'ẻ', 'ẹ', 'ệ', 'g', 'h', 'i', 'ì', 'í', 'ĩ', 'ỉ', 'ị', 'k', 'l', 'm', 'n', 'o', 'ò', 'ó', 'ô', 'ồ', 'ố', 'ỗ', 'ổ', 'õ', 'ỏ', 'ơ', 'ờ', 'ớ', 'ỡ', 'ở', 'ợ', 'ọ', 'ộ', 'p', 'q', 'r', 's', 't', 'u', 'ù', 'ú', 'ũ', 'ủ', 'ư', 'ừ', 'ứ', 'ữ', 'ử', 'ự', 'ụ', 'v', 'x', 'y', 'ỳ', 'ý', 'ỹ', 'ỷ', 'ỵ', 'đ']
  listnum = [i for i in range(0,89)]
  if (num >= 0 and num <= 88):
    k = listnum.index(num)
    return listchar[k]
  else:
    return -1

In [23]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras import backend as K
from keras.utils import np_utils
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler


scaler = MinMaxScaler()
scaler.fit(data_train)
data_train = scaler.transform(data_train)
data_test = scaler.transform(data_test)

data_train = data_train.reshape((data_train.shape[0], 28, 28, 1)).astype('float32')
data_test = data_test.reshape((data_test.shape[0], 28, 28, 1)).astype('float32')

print(data_train.shape) 

label_train = np_utils.to_categorical(label_train)
label_test = np_utils.to_categorical(label_test)

print(label_test.shape)

num_classes = label_test.shape[1]

print(num_classes)

(19229, 28, 28, 1)
(4449, 89)
89


In [26]:
model = Sequential()

model.add(Conv2D(64, (5, 5), input_shape=(28, 28, 1), activation='relu', data_format="channels_last", padding="same"))
model.add(Conv2D(64, (5, 5), input_shape=(28, 28, 1), activation='relu', data_format="channels_last", padding="same"))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu', data_format="channels_last", padding="same"))
model.add(Conv2D(128, (3, 3), activation='relu', data_format="channels_last", padding="same"))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Dropout(0.2))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(data_train, label_train, validation_data=(data_test, label_test), epochs=20, batch_size=256, verbose=2)

# Final evaluation of the model
scores = model.evaluate(data_test, label_test, verbose=0)
print("CNN Accuracy: %.2f%%" % (scores[1]*100))

Epoch 1/20
76/76 - 377s - loss: 2.6473 - accuracy: 0.3635 - val_loss: 2.0452 - val_accuracy: 0.4473
Epoch 2/20
76/76 - 359s - loss: 0.9578 - accuracy: 0.7228 - val_loss: 1.4814 - val_accuracy: 0.6017
Epoch 3/20
76/76 - 360s - loss: 0.5526 - accuracy: 0.8376 - val_loss: 1.3453 - val_accuracy: 0.6521
Epoch 4/20
76/76 - 359s - loss: 0.3600 - accuracy: 0.8941 - val_loss: 1.4026 - val_accuracy: 0.6662
Epoch 5/20
76/76 - 359s - loss: 0.2649 - accuracy: 0.9197 - val_loss: 1.4727 - val_accuracy: 0.6669
Epoch 6/20
76/76 - 359s - loss: 0.1763 - accuracy: 0.9451 - val_loss: 1.6360 - val_accuracy: 0.6811
Epoch 7/20
76/76 - 359s - loss: 0.1423 - accuracy: 0.9552 - val_loss: 1.6247 - val_accuracy: 0.6815
Epoch 8/20
76/76 - 360s - loss: 0.0952 - accuracy: 0.9687 - val_loss: 1.8737 - val_accuracy: 0.6678
Epoch 9/20
76/76 - 358s - loss: 0.0822 - accuracy: 0.9727 - val_loss: 1.9222 - val_accuracy: 0.6826
Epoch 10/20
76/76 - 359s - loss: 0.0677 - accuracy: 0.9772 - val_loss: 1.9631 - val_accuracy: 0.6912